# Covid 19 Prediction, Covid 19 patient's admission to the ward using Supervised Machine Learning.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import Libraries

In [2]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
#from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import ExtraTreesClassifier
import warnings
warnings.filterwarnings('ignore')

Load data

In [3]:
data = pd.read_csv("corona_tested_individuals_subset_ver_00184.csv")

Getting to know the data!

In [4]:
data.info()
data.describe()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101796 entries, 0 to 101795
Data columns (total 10 columns):
test_date              101796 non-null object
cough                  101796 non-null int64
fever                  101796 non-null int64
sore_throat            101796 non-null int64
shortness_of_breath    101796 non-null int64
head_ache              101796 non-null int64
corona_result          101796 non-null object
age_60_and_above       87260 non-null object
gender                 76816 non-null object
test_indication        101796 non-null object
dtypes: int64(5), object(5)
memory usage: 7.8+ MB


,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2022-03-03,0,0,0,0,0,חיובי,No,נקבה,Other
1,2022-03-03,0,0,0,0,0,חיובי,No,זכר,Other
2,2022-03-03,0,0,0,0,0,חיובי,No,זכר,Other
3,2022-03-03,0,0,0,0,0,חיובי,No,נקבה,Other
4,2022-03-03,0,0,0,0,0,חיובי,No,נקבה,Abroad


Feature Engineering!

In [5]:
data.columns = [x.lower().strip().replace(' ','_') for x in data.columns]

In [6]:
def miss_data(x):
    total = x.isnull().sum()
    percent = (x.isnull().sum()/x.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(x[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [7]:
miss_data(data)

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
Total,0,0,0,0,0,0,0,14536,24980,0
Percent,0,0,0,0,0,0,0,14.2795,24.5393,0
Types,object,int64,int64,int64,int64,int64,object,object,object,object


In [8]:
for x in data.columns:
    if data[x].dtype=='float16' or  data[x].dtype=='float32' or  data[x].dtype=='float64':
        data[x].fillna(data[x].mean())

data = data.fillna(-999)

for y in data.columns:
    if data[y].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(data[y].values))
        data[y] = lbl.transform(list(data[y].values))

In [9]:
threshold = 0.92

corr_matrix = data.corr().abs()
corr_matrix.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
test_date,1.000000,0.023398,0.009282,0.017838,0.007348,0.017131,0.123399,0.082412,0.016014,0.012312
cough,0.023398,1.000000,0.345586,0.442269,0.141027,0.408612,0.280198,0.134683,0.081165,0.252409
fever,0.009282,0.345586,1.000000,0.282340,0.079471,0.307467,0.185621,0.079538,0.051860,0.166844
sore_throat,0.017838,0.442269,0.282340,1.000000,0.088577,0.433233,0.244814,0.103049,0.072203,0.218151
shortness_of_breath,0.007348,0.141027,0.079471,0.088577,1.000000,0.101546,0.061607,0.031359,0.023213,0.060243


In [10]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
test_date,NaN,0.023398,0.009282,0.017838,0.007348,0.017131,0.123399,0.082412,0.016014,0.012312
cough,NaN,NaN,0.345586,0.442269,0.141027,0.408612,0.280198,0.134683,0.081165,0.252409
fever,NaN,NaN,NaN,0.282340,0.079471,0.307467,0.185621,0.079538,0.051860,0.166844
sore_throat,NaN,NaN,NaN,NaN,0.088577,0.433233,0.244814,0.103049,0.072203,0.218151
shortness_of_breath,NaN,NaN,NaN,NaN,NaN,0.101546,0.061607,0.031359,0.023213,0.060243


In [11]:
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))
dataset = data.drop(columns = to_drop)
print('Data shape: ', data.shape)
print('Size of the data', data.shape)

There are 0 columns to remove.
Data shape:  (101796, 10)
Size of the data (101796, 10)


In [12]:
data_missing = (data.isnull().sum() / len(data)).sort_values(ascending = False)
data_missing.head()

test_indication     0.0
gender              0.0
age_60_and_above    0.0
corona_result       0.0
head_ache           0.0
dtype: float64

In [13]:
data_missing_ = data_missing.index[data_missing > 0.85]
all_missing = list(set(data_missing_))

In [14]:
dataset = dataset.drop(columns = all_missing)

In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101796 entries, 0 to 101795
Data columns (total 10 columns):
test_date              101796 non-null int64
cough                  101796 non-null int64
fever                  101796 non-null int64
sore_throat            101796 non-null int64
shortness_of_breath    101796 non-null int64
head_ache              101796 non-null int64
corona_result          101796 non-null int64
age_60_and_above       101796 non-null int64
gender                 101796 non-null int64
test_indication        101796 non-null int64
dtypes: int64(10)
memory usage: 7.8 MB


In [16]:
cols = [x for x in dataset.columns if x not in ['patient_id','sars-cov-2_exam_result', 'patient_addmited_to_regular_ward_(1=yes,_0=no)', 'patient_addmited_to_semi-intensive_unit_(1=yes,_0=no)', 'patient_addmited_to_intensive_care_unit_(1=yes,_0=no)']]

In [17]:
new_df = dataset[cols]

In [18]:
new_df

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,14,0,0,0,0,0,1,1,2,2
1,14,0,0,0,0,0,1,1,1,2
2,14,0,0,0,0,0,1,1,1,2
3,14,0,0,0,0,0,1,1,2,2
4,14,0,0,0,0,0,1,1,2,0
5,14,0,0,0,0,0,1,1,2,0
6,14,0,0,0,0,0,1,1,2,2
7,14,1,1,1,0,1,1,1,2,2
8,14,1,1,0,0,0,1,2,2,1
9,14,0,0,0,0,0,1,1,1,2


Data Split

In [23]:
X = new_df
y = 'test_indication'

In [25]:
'''X = new_df'''
'''y = pd.read_csv('corona_tested_individuals_subset_ver_00184.csv')'''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=101)

ValueError: Found input variables with inconsistent numbers of samples: [101796, 15]

Feature Importance using Extra Trees Classifier

In [ ]:
feat_head = feat_importances.head(10)
feat_head.index

In [ ]:
X = new_df[feat_head.index]
y = dataset['sars-cov-2_exam_result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 101)

ML Models

In [ ]:
accuracy_lst =[]

def model_assess(model, name='Default'):
    model.fit(X_train, y_train)
    prds = model.predict(X_test)
    model_acc = accuracy_score(y_test, prds)
    accuracy_lst.append(100*model_acc)
    print('---', name, '---', '\n',
          confusion_matrix(y_test, prds), '\n',
          'Accuracy:', (accuracy_score(y_test, prds)), '\n',
          'Classification Report:', (classification_report(y_test, prds)))

In [ ]:
# Logistic Regression
lg = LogisticRegression()
model_assess(lg, 'Logistic Regression')

# Decision Tree
tree = DecisionTreeClassifier()
model_assess(tree, 'Decission Trees')

# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, 'Random Forest')

# SVM
svm = SVC()
model_assess(svm, 'SVM')

# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, name='KNN')

# XGBOOST
'''
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_assess(xgb, 'XGBoost')
'''
# Neural Network
nn = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(150, 10), random_state=1)
model_assess(nn, 'Neural Nets')

In [ ]:
model_list = ['Logistic Regression', 'DT', 'Random Forest', 'SVM', 'KNearestNeighbours', 'NN']

In [ ]:
plt.rcParams['figure.figsize']=20,8
sns.set_style('darkgrid')
ax = sns.barplot(x=model_list, y = accuracy_lst, palette = "coolwarm", saturation =2.0)
plt.xlabel('Classifier Models', fontsize = 20 )
plt.ylabel('% of Accuracy', fontsize = 20)
plt.title('Accuracy of different Classifier Models', fontsize = 20)
plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 12)
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax.annotate(f'{round(height,2)}%', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
plt.show()

In [ ]:
cross_acc = []

ca_lg = cross_val_score(lg, X_train, y_train, scoring='accuracy')
ca_lg = ca_lg.mean()
cross_acc.append(100*ca_lg)

ca_tree = cross_val_score(tree, X_train, y_train, scoring='accuracy')
ca_tree = ca_tree.mean()
cross_acc.append(100*ca_tree)

ca_rforest = cross_val_score(rforest, X_train, y_train, scoring='accuracy')
ca_rforest = ca_rforest.mean()
cross_acc.append(100*ca_rforest)

ca_svm = cross_val_score(svm, X_train, y_train, scoring='accuracy')
ca_svm = ca_svm.mean()
cross_acc.append(100*ca_svm)

ca_knn = cross_val_score(knn, X_train, y_train, scoring='accuracy')
ca_knn = ca_knn.mean()
cross_acc.append(100*ca_knn)

ca_xgb = cross_val_score(xgb, X_train, y_train, scoring='accuracy')
ca_xgb = ca_xgb.mean()
cross_acc.append(100*ca_xgb)

In [ ]:
ca_nn = cross_val_score(nn, X_train, y_train, scoring='accuracy')
ca_nn = ca_nn.mean()
cross_acc.append(100*ca_nn)

In [ ]:
plt.rcParams['figure.figsize']=20,8
sns.set_style('darkgrid')
ax = sns.barplot(x=model_list, y=cross_acc, palette = "rocket", saturation =2.0)
plt.xlabel('Classifier Models', fontsize = 20 )
plt.ylabel('Cross validation Accuracy', fontsize = 20)
plt.title('Accuracy of different Classifier Models', fontsize = 20)
plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 12)
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax.annotate(f'{round(height,2)}%', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
plt.show()

Admission to ward (Covid-19 Patients) 

In [ ]:
covid_positive = dataset[dataset['sars-cov-2_exam_result'] == 1]

In [ ]:
admission = []  

def multiclass_target(row):
    check = 0
    check += 1 if (row['patient_addmited_to_regular_ward_(1=yes,_0=no)'] == 1) else 0
    check += 2 if (row['patient_addmited_to_semi-intensive_unit_(1=yes,_0=no)'] == 1) else 0
    check += 3 if (row['patient_addmited_to_intensive_care_unit_(1=yes,_0=no)'] == 1) else 0
    row['target'] = check
    return row

data_adm = covid_positive.apply(multiclass_target, axis=1)
data_adm

In [ ]:
X = data_adm[feat_head.index]
y = data_adm['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

ML Models for Ward Prediction!

In [ ]:
acc_lst = []

def model_assess(model, name='Default'):
    model.fit(X_train, y_train)
    prds = model.predict(X_test)
    model_acc = accuracy_score(y_test, prds)
    acc_lst.append(100*model_acc)
    print('---', name, '---', '\n',
          confusion_matrix(y_test, prds), '\n',
          'Accuracy:', (accuracy_score(y_test, prds)), '\n',
          'Classification Report:', (classification_report(y_test, prds)))

In [ ]:
# Logistic Regression
lg = LogisticRegression()
model_assess(lg, 'Logistic Regression')

# Decision Tree
tree = DecisionTreeClassifier()
model_assess(tree, 'Decission Trees')

# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, 'Random Forest')

# SVM
svm = SVC()
model_assess(svm, 'SVM')

# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, name='KNN')

# XGBOOST
# xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
# model_assess(xgb, 'XGBoost')

# Neural Network
nn = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(150, 10), random_state=1)
model_assess(nn, 'Neural Nets')

In [ ]:
from sklearn import preprocessing
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
train_feature = minmax_scale.fit_transform(df_train_feature)
test_feature = minmax_scale.fit_transform(df_test_feature)

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

model = Sequential() 

model.add(Dense(units=5000, 
                input_dim=46, 
                kernel_initializer='uniform', 
                activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1500,  
                kernel_initializer='uniform', 
                activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,
                kernel_initializer='uniform', 
                activation='sigmoid'))

print(model.summary())
model.compile(loss='binary_crossentropy',   
              optimizer='adam', metrics=['accuracy'])

train_history = model.fit(x=train_feature, y=train_label,  
                          validation_split=0.2, epochs=20, 
                          batch_size=200, verbose=2) 

show_train_history(train_history,'acc','val_acc')
show_train_history(train_history,'loss','val_loss')

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

model = Sequential() 

model.add(Dense(units=5000, 
                input_dim=46, 
                kernel_initializer='uniform', 
                activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1500,  
                kernel_initializer='uniform', 
                activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,
                kernel_initializer='uniform', 
                activation='sigmoid'))

print(model.summary())
model.compile(loss='binary_crossentropy',   
              optimizer='adam', metrics=['accuracy'])

train_history = model.fit(x=train_feature, y=train_label,  
                          validation_split=0.2, epochs=20, 
                          batch_size=200, verbose=2) 

show_train_history(train_history,'acc','val_acc')
show_train_history(train_history,'loss','val_loss')

In [ ]:
plt.rcParams['figure.figsize']=20,8
sns.set_style('darkgrid')
ax = sns.barplot(x=model_list, y = acc_lst, palette = "coolwarm", saturation =2.0)
plt.xlabel('Classifier Models', fontsize = 20 )
plt.ylabel('% of Accuracy', fontsize = 20)
plt.title('Accuracy of different Classifier Models', fontsize = 20)
plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 12)
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax.annotate(f'{round(height,2)}%', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
plt.show()